In [0]:
%tensorflow_version 1.x

In [0]:
import struct
import numpy as np
import sys

def write_htk_user_feat(x, name='filename'):
  
    default_period = 100000 # assumes 0.010 ms frame shift
    num_dim = x.shape[0]
    num_frames = x.shape[1]
    hdr = struct.pack(
        '>iihh',  # the beginning '>' says write big-endian
        num_frames,  # nSamples
        default_period,  # samplePeriod
        4*num_dim,  # 2 floats per feature
        9)  # user features

    out_file = open(name, 'wb')
    out_file.write(hdr)

    for t in range(0, num_frames):
        frame = np.array(x[:,t],'f')
        if sys.byteorder == 'little':
            frame.byteswap(True)
        frame.tofile(out_file)

    out_file.close()

def read_htk_user_feat(name='filename'):
    f = open(name,'rb')
    hdr = f.read(12)
    num_samples, samp_period, samp_size, parm_kind = struct.unpack(">IIHH", hdr)
    if parm_kind != 9:
        raise RuntimeError('feature reading code only validated for USER feature type for this lab. There is other publicly available code for general purpose HTK feature file I/O\n')

    num_dim = samp_size//4

    feat = np.zeros([num_samples, num_dim],dtype=float)
    for t in range(num_samples):
        feat[t,:] = np.array(struct.unpack('>' + ('f' * num_dim), f.read(samp_size)),dtype=float)

    return feat


def write_ascii_stats(x,name='filename'):
    out_file = open(name,'w')
    for t in range(0, x.shape[0]):
        out_file.write("{0}\n".format(x[t]))
    out_file.close()

def MyMeanVarNorm(feature_mean_file, feature_inv_stddev_file):
    m = C.reshape(load_ascii_vector(feature_mean_file,'feature_mean'), shape=(1, feature_dim))
    s = C.reshape(load_ascii_vector(feature_inv_stddev_file,'feature_invstddev'), shape=(1,feature_dim))
    def _func(operand):
        return C.reshape(C.element_times(C.reshape(operand,shape=(1+context[0]+context[1], feature_dim)) - m, s), shape=operand.shape)
    return _func

def load_ascii_vector(file_name, var_name=None):
    x = np.asarray(np.loadtxt(file_name), dtype=np.float32)
    return x


In [32]:
#write_htk_user_feat(None, name="/content/drive/My Drive/DataSetSp/feat/test.feat")
from __future__ import print_function
import numpy as np
import keras
import tensorflow
from keras.preprocessing import sequence
#from keras.models import Sequential,Model
#from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,InputLayer,TimeDistributed,Multiply,Input,Lambda,Masking,CuDNNLSTM
from tensorflow.keras.backend import ctc_batch_cost

print(keras.__version__)
print(tensorflow.__version__)




2.3.1
1.15.2


In [33]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)






Mounted at /content/drive


In [0]:
from keras.utils import to_categorical, np_utils

model_type='dev_train/217'
uniqueLabels=set([])

In [0]:
#!cp -r  "/content/drive/My Drive/DataSetSp/shehab/dev_train/217/feat" "feat"

In [0]:
!cp  "/content/drive/My Drive/DataSetSp/shehab/dev_train/217/data.zip"    ./    
#!cp  Data.zip "/content/drive/My Drive/DataSetSp/shehab/dev_train/217/Data2.zip"   

In [0]:

!unzip -j data.zip -d ./feat
#!zip -r  "/content/drive/My Drive/DataSetSp/shehab/dev_train/217/Data.zip" "/content/drive/My Drive/DataSetSp/shehab/dev_train/217/feat"
#!ls "/content/drive/My Drive/DataSetSp/shehab/dev_train/217"
# change it inly in read data function

Streaming output truncated to the last 5000 lines.
  inflating: ./feat/7367-86737-0088.feat  
  inflating: ./feat/7367-86737-0080.feat  
  inflating: ./feat/7367-86737-0097.feat  
  inflating: ./feat/7367-86737-0027.feat  
  inflating: ./feat/7367-86737-0077.feat  
  inflating: ./feat/7367-86737-0004.feat  
  inflating: ./feat/7367-86737-0105.feat  
  inflating: ./feat/7367-86737-0064.feat  
  inflating: ./feat/7367-86737-0056.feat  
  inflating: ./feat/7367-86737-0003.feat  
  inflating: ./feat/7367-86737-0028.feat  
  inflating: ./feat/7367-86737-0033.feat  
  inflating: ./feat/7367-86737-0062.feat  
  inflating: ./feat/7367-86737-0017.feat  
  inflating: ./feat/7367-86737-0059.feat  
  inflating: ./feat/7367-86737-0043.feat  
  inflating: ./feat/7367-86737-0022.feat  
  inflating: ./feat/7367-86737-0047.feat  
  inflating: ./feat/7367-86737-0002.feat  
  inflating: ./feat/7367-86737-0094.feat  
  inflating: ./feat/7367-86737-0112.feat  
  inflating: ./feat/7367-86737-0006.feat  
  i

In [0]:
! ls

data.zip  drive  feat  sample_data


In [0]:
# cut texts after this number of words
# (among top max_features most common words)

labelFile = "/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/alignment"

invStd=load_ascii_vector("/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/feat_invstddev.ascii")
mean=load_ascii_vector("/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/feat_mean.ascii")



def createExample(file):
  inv=np.array(invStd)
  m=np.array(mean)


  i = 0
  labels=file[2]
  fileName=file[0]
  j=0

  try:
    di = '/'.join(fileName.split('-')[0:2])+"/"
    #print("/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/feat/"+fileName+".feat")
    #features=read_htk_user_feat("/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/feat/"+di+fileName+".feat")
    features=read_htk_user_feat("feat/"+fileName+".feat")
    #print(fileName)
    #print("features size (",features.shape[0]," , ",features.shape[1],")")
    #print("\n\n",features,'\n\n')
    #print(labels)
    x_list=[]
    y_list=[]

    start=0
    end=features.shape[0]
    count = 0

    #print(features.shape)
    #print(features)
        
    for  i in range(0,len(labels)):
        #print(i,'-----'*5)
        l=labels[i].strip().split()
        number_of_frames=int(l[1])
        label=int(l[0])


        feat=np.array(features[start:start+number_of_frames])


        feat=feat-m

        feat=np.multiply(inv,feat)


        x_list=x_list+feat.tolist()
        y=np.full((feat.shape[0]),label)
        y_list=y_list+y.tolist()




        start=start+number_of_frames
        #print(start)
        count=count+number_of_frames

    #print(len(x_list),len(y_list),count,features.shape[0],len(x_list[0]),type(y_list[0]))
    return x_list,y_list
  except IOError :
    print("file scipped")
    return None,None


      
        



def readData(fms):   ## fms file mapping sorted
    x_train_list=[]
    y_train_list=[]
    c=0
    count_notUsed = 0
    count_used=0
    for file in fms:
        #print(file)
        x_train,y_train = createExample(file)

        if (x_train is None and y_train is None ):
          count_notUsed+=1
        else:
          x_train_list.append(x_train)
          y_train_list.append(y_train)
          count_used+=1

        #print(c,'\n')
        c=c+1


        
    #print("number of files not used is ", count_notUsed)
    #print("number of files  used is ", count_used)
    return x_train_list,y_train_list  



    

    



In [0]:
def get_labels(labelFile):
  
  file_with_mapping=[]
  print("a")
  with open(labelFile) as f:
    labels = f.readlines()
    labels = [x.strip() for x in labels]
    
    for label in labels:
      frames=label.split(";")
      #print(frames)
      zero=frames[0]
      zeros=zero.split()
      file_name=zeros[0]

      frames[0]=' '.join(zeros[1:3])

      print(file_name)
      lenn=0

      for frame in frames:
        arr= frame.split()
        uniqueLabels.add(int(arr[0]))
        lenn=lenn+int(arr[1])

      file_with_mapping.append((file_name,lenn,frames))


  return file_with_mapping

        



In [36]:

file_with_mapping=get_labels(labelFile)

len(file_with_mapping)
file_with_mapping[0]
len(uniqueLabels)

import numpy as np
from sklearn.model_selection import train_test_split

train = file_with_mapping
train, test, _, _ = train_test_split(file_with_mapping, file_with_mapping, test_size=0.06, random_state=42)
print(len(train),len(test))


  

Streaming output truncated to the last 5000 lines.
7367-86737-0079
7367-86737-0080
7367-86737-0081
7367-86737-0082
7367-86737-0083
7367-86737-0084
7367-86737-0085
7367-86737-0086
7367-86737-0087
7367-86737-0088
7367-86737-0089
7367-86737-0090
7367-86737-0091
7367-86737-0092
7367-86737-0093
7367-86737-0094
7367-86737-0095
7367-86737-0096
7367-86737-0097
7367-86737-0098
7367-86737-0099
7367-86737-0100
7367-86737-0101
7367-86737-0102
7367-86737-0103
7367-86737-0104
7367-86737-0105
7367-86737-0106
7367-86737-0107
7367-86737-0108
7367-86737-0109
7367-86737-0110
7367-86737-0111
7367-86737-0112
7367-86737-0113
7367-86737-0114
7367-86737-0115
7367-86737-0116
7367-86737-0117
7367-86737-0118
7402-59171-0000
7402-59171-0001
7402-59171-0002
7402-59171-0003
7402-59171-0004
7402-59171-0005
7402-59171-0006
7402-59171-0007
7402-59171-0008
7402-59171-0009
7402-59171-0010
7402-59171-0011
7402-59171-0012
7402-59171-0013
7402-59171-0014
7402-59171-0015
7402-59171-0016
7402-59171-0017
7402-59171-0018
7402-

In [37]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import pickle
def encodingLabels(uniqueLabels):
  labels=sorted(uniqueLabels)
  print(len(labels))
  encoder = LabelEncoder()
  encoder.fit(labels)
  print(encoder.classes_)

  pickle.dump(encoder,open("/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/label_encoder","wb"))



encodingLabels(uniqueLabels)
encoder = pickle.load( open( "/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/label_encoder", "rb" ) )



217
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217]


In [38]:
len(uniqueLabels)

217

In [39]:
import random
def divide_to_batches(fms,variance=10,limit=100*1000):
  num_samples = len(fms)
  fms=sorted(fms,key=lambda x: x[1])
  start=0
  end =0
  result=[]
  while start <num_samples:
    end=start
    while ((end < num_samples) and (fms[end][1]-fms[start][1] <= variance) and  ((end-start)*fms[start][1]<limit)):
      end=end+1
    if(end >=num_samples):
        end=num_samples
    
    result.append((start,end))
    start=end

  return result


def generator(file_with_mapping,encoder,uniqueLabels,variance=10,limit=100*1000):

    labels=sorted(uniqueLabels)
  
    num_samples = len(file_with_mapping)

    file_with_mapping_sorted=sorted(file_with_mapping,key=lambda x: x[1])

    #for m in file_with_mapping_sorted:
    #  print(m[1])

    fms=file_with_mapping_sorted
    batches_indices_original=divide_to_batches(file_with_mapping,variance,limit)
    batches_indices=batches_indices_original
    while True: # Loop forever so the generator never terminates
          random.shuffle(batches_indices)
          for indices in batches_indices:  
             
              batch = fms[indices[0]:indices[1]]
              #print(len(batch))
              
              x_batch,y_batch=readData(batch)

              truncating_len=len(x_batch[0])


              encoded_y = [ encoder.transform(y) for y in y_batch]
              y_batch = [to_categorical(y,num_classes=len(labels)) for y in encoded_y]

              #print(len(y_batch))
              #print(len(y_batch[0]))
              #print(len(y_batch[0][0]))

              #print(len(x_batch))
              #print(len(x_batch[0]))
              #print(len(x_batch[0][0]))

              x_batch=sequence.pad_sequences(x_batch,maxlen=truncating_len,dtype='float32',truncating='post')

              y_batch=sequence.pad_sequences(y_batch,maxlen=truncating_len,dtype='float32',truncating='post')




              indices = np.arange(len(x_batch))
              np.random.shuffle(indices)

              x_batch=  x_batch[indices] 
              y_batch = y_batch[indices]



              x_batch=x_batch.reshape(len(x_batch),-1,40)
              y_batch=y_batch.reshape(len(y_batch),-1,len(labels))


              #z = np.zeros((y_batch.shape[0],y_batch.shape[1],1), dtype='float32')
              #y_batch=np.concatenate((z,y_batch), axis=2)


            
    
   
              yield  x_batch,y_batch

          batches_indices=batches_indices_original

train_generator=generator(train,encoder,uniqueLabels,1)
val_generator=generator(test,encoder,uniqueLabels,10)

def calc_steps_epoch(fms,variance=10,limit=100*1000):
 return len(divide_to_batches(fms,variance,limit))



print(calc_steps_epoch(train,variance=1))

      

          

867


In [0]:
def ctc_find_eos(y_true, y_pred):

    #convert y_pred from one-hot to label indices
    y_true = K.argmax(y_true, axis=-1)
  
    #lengths
    true_lengths = 1 + K.argmax(occurrence_weights * is_eos_true, axis=1)

    #reshape
    true_lengths = K.reshape(true_lengths, (-1,1))
    pred_lengths = K.reshape(true_lengths, (-1,1))

    return K.ctc_batch_cost(y_true, y_pred, pred_lengths, true_lengths)

In [0]:
batch_size = 32
#print(prior)
import keras.backend as K
import tensorflow as tf


#prior1=K.constant(prior,dtype='float32')
#prior2 = Input(tensor=prior1,shape=(None,118))
inputt=Input(shape=(None,40))
#inputt2=Masking(mask_value=0,input_shape=(None,40))(inputt)
l1=Bidirectional(CuDNNLSTM(512,return_sequences=True))(inputt)
l2=Bidirectional(CuDNNLSTM(512,return_sequences=True))(l1)
l4=TimeDistributed(Dense(len(uniqueLabels), activation='softmax'))(l2)


model = Model(inputs=[inputt], outputs=l4)

#from keras import optimizers

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

checkpoint_path = "/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/model_epoch/cp-{epoch:04d}.ckpt"

from keras.callbacks import ModelCheckpoint
# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1,
    period=1)



In [0]:


# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=["categorical_accuracy"])
#model.compile(loss=ctc_loss,
#              optimizer='adam',metrics=["categorical_accuracy"])

print(model.summary())


model.fit_generator(generator=train_generator,
                    validation_data=val_generator,
                    epochs=30,
                    steps_per_epoch=calc_steps_epoch(train,variance=1),
                    validation_steps= calc_steps_epoch(test,variance=10),
                    callbacks=[cp_callback])

#print('Train...')
#model.fit(x_train, y_train,
#          batch_size=40,
#          epochs=20,
#          validation_data=[x_test, y_test],
#          callbacks=[cp_callback])

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, None, 40)          0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, None, 1024)        2269184   
_________________________________________________________________
bidirectional_14 (Bidirectio (None, None, 1024)        6299648   
_________________________________________________________________
bidirectional_15 (Bidirectio (None, None, 512)         2625536   
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 217)         111321    
Total params: 11,305,689
Trainable params: 11,305,689
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
867/867 [==============================] - 2836s 3s/step - loss: 0.5892 - categorical_accuracy: 0

In [0]:
#model.save_weights("/content/drive/My Drive/DataSetSp/model2")
print(len(x_train),len(x_train[1]))


In [0]:
#model.save("/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/model_90")

#Load partly trained model
from keras.models import load_model
model2 = load_model("/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/cp-0030.ckpt")



In [41]:


#results = model2.evaluate_generator(train_generator,steps=calc_steps_epoch(train))
#print('train loss, train acc:', results)

results = model2.evaluate_generator(val_generator,steps=calc_steps_epoch(test,variance=10))
print('test loss, test acc:', results)


test loss, test acc: [0.3556792140007019, 0.9044780731201172]


In [43]:
import glob
import random

mylist = [f for f in glob.glob("/content/drive/My Drive/DataSetSp/shehab/217/feat/*.feat")]
mylist = [m  for m in mylist if '(' not in m]

# mylist=["/content/drive/My Drive/DataSetSp/shehab/217/feat/174-84280-0001.feat",
# "/content/drive/My Drive/DataSetSp/shehab/217/feat/2035-147960-0011.feat",
# "/content/drive/My Drive/DataSetSp/shehab/217/feat/2078-142845-0000.feat",
# "/content/drive/My Drive/DataSetSp/shehab/217/feat/2428-83705-0006.feat",
# "/content/drive/My Drive/DataSetSp/shehab/217/feat/3170-137482-0041.feat",
# "/content/drive/My Drive/DataSetSp/shehab/217/feat/5338-24615-0000.feat",
# "/content/drive/My Drive/DataSetSp/shehab/217/feat/6241-61943-0026.feat",
# "/content/drive/My Drive/DataSetSp/shehab/217/feat/6295-64301-0007.feat",
# "/content/drive/My Drive/DataSetSp/shehab/217/feat/777-126732-0030.feat",
# "/content/drive/My Drive/DataSetSp/shehab/217/feat/777-126732-0041.feat"]
print(len(mylist))

#### showay 7aga * 40
#### 
cl=Classifier("/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/cp-0030.ckpt")
for i in range(0,100):
  r=random.randrange(0, len(mylist))
  r=mylist[r]
  features=read_htk_user_feat(r)
  feat=[]
  feat.append(features)
  
  #sample=np.array(feat)
  print("sample.shape",sample.shape)
  print("features",features.shape)
  #sample=sample.reshape(1,features.shape[0],features.shape[1])
  #print("sample.shape",sample.shape)
  #pred=model2.predict(sample)
  #pred=pred[0]
  pred=cl.eval(features)
  print(pred.shape)
  print(r)
  r=r.split('/')[-1]
  r=r.replace('feat','txt')
  print(r)

  #np.savetxt("/content/drive/My Drive/DataSetSp/shehab/"+model_type+"/ex/second/"+r,pred)

  

2703
sample.shape (1, 628, 40)
features (474, 40)
(474, 217)
/content/drive/My Drive/DataSetSp/shehab/217/feat/1272-135031-0003.feat
1272-135031-0003.txt
sample.shape (1, 628, 40)
features (305, 40)
(305, 217)
/content/drive/My Drive/DataSetSp/shehab/217/feat/1462-170142-0036.feat
1462-170142-0036.txt
sample.shape (1, 628, 40)
features (398, 40)
(398, 217)
/content/drive/My Drive/DataSetSp/shehab/217/feat/2428-83699-0028.feat
2428-83699-0028.txt
sample.shape (1, 628, 40)
features (933, 40)
(933, 217)
/content/drive/My Drive/DataSetSp/shehab/217/feat/6313-66125-0023.feat
6313-66125-0023.txt
sample.shape (1, 628, 40)
features (449, 40)
(449, 217)
/content/drive/My Drive/DataSetSp/shehab/217/feat/652-129742-0020.feat
652-129742-0020.txt
sample.shape (1, 628, 40)
features (554, 40)
(554, 217)
/content/drive/My Drive/DataSetSp/shehab/217/feat/7976-105575-0005.feat
7976-105575-0005.txt
sample.shape (1, 628, 40)
features (603, 40)
(603, 217)
/content/drive/My Drive/DataSetSp/shehab/217/feat/5

KeyboardInterrupt: ignored

In [0]:
import numpy as np
from keras.models import load_model


class Classifier:
  def __init__(self,model_filename: str):
    self.model = None
    try:
      self.model = load_model(model_filename)
    except:
      raise Exception("Model didn't load successfully")

  def load_model(self,model_filename):
    try:
      self.model = load_model(model_filename)
    except:
      raise Exception("Model didn't load successfully")


  def eval(self, features):
    if(self.model):
      sample=features.reshape(1,features.shape[0],features.shape[1])
      pred=self.model.predict(sample)
      return pred[0]
    else:
      raise  Exception("Model isn't loaded yet")


class Classifier:
  def __init__(self,model_filename: str):
    self.model = None
    try:
      self.model = load_model(model_filename)
    except:
      raise Exception("Model didn't load successfully")

  def load_model(self,model_filename):
    try:
      self.model = load_model(model_filename)
    except:
      raise Exception("Model didn't load successfully")


  def eval(self, features):
    if(self.model):
      sample=features.reshape(1,features.shape[0],features.shape[1])
      pred=self.model.predict(sample)
      return pred[0]
    else:
      raise  Exception("Model isn't loaded yet")

In [0]:
!zip -r -j  "/content/drive/My Drive/DataSetSp/shehab/dev_train/217/ex/train.zip" "/content/drive/My Drive/DataSetSp/shehab/dev_train/217/ex/first"
#!zip -r -j  "/content/drive/My Drive/DataSetSp/shehab/dev_train/217/ex/dev.zip" "/content/drive/My Drive/DataSetSp/shehab/dev_train/217/ex/second"


  adding: 40-121026-0006.txt (deflated 55%)
  adding: 6181-216552-0039.txt (deflated 55%)
  adding: 1263-138246-0017.txt (deflated 55%)
  adding: 2817-142371-0001.txt (deflated 55%)
  adding: 4830-25904-0007.txt (deflated 55%)
  adding: 7178-34644-0072.txt (deflated 55%)
  adding: 4481-17498-0022.txt (deflated 55%)
  adding: 4267-78186-0024.txt (deflated 55%)
  adding: 1743-142914-0012.txt (deflated 55%)
  adding: 7113-86041-0088.txt (deflated 55%)
  adding: 2989-138035-0027.txt (deflated 55%)
  adding: 2384-152900-0005.txt (deflated 55%)
  adding: 4297-13006-0026.txt (deflated 55%)
  adding: 2764-36617-0033.txt (deflated 55%)
  adding: 4441-76262-0026.txt (deflated 55%)
  adding: 2691-156745-0020.txt (deflated 55%)
  adding: 4014-186183-0029.txt (deflated 55%)
  adding: 3857-180923-0009.txt (deflated 55%)
  adding: 5456-62014-0005.txt (deflated 55%)
  adding: 2989-138035-0022.txt (deflated 55%)
  adding: 441-128988-0024.txt (deflated 55%)
  adding: 1363-139304-0015.txt (deflated 55%)
